See [schema](https://oss-augur.readthedocs.io/en/dev/_images/schema.png) for details.

In [1]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as s
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = s.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [3]:
table_query = f"""
SELECT
   tablename
FROM
   pg_catalog.pg_tables
WHERE
   schemaname = 'augur_data';
    """
tables = pd.read_sql_query(table_query, con=engine)
print(tables)

                   tablename
0                _git_census
1       contributors_aliases
2   contributor_affiliations
3       dm_repo_group_annual
4     pull_request_assignees
5        pull_request_events
6         pull_request_teams
7                repo_groups
8         repo_test_coverage
9            repos_fetch_log
10                  settings
11             unknown_cache
12               utility_log
13           working_commits
14                repo_stats
15        commit_comment_ref
16              analysis_log
17      chaoss_metric_status
18            commit_parents
19      contributors_history
20            dm_repo_annual
21                   commits
22              contributors
23     dm_repo_group_monthly
24      dm_repo_group_weekly
25           dm_repo_monthly
26            dm_repo_weekly
27                   exclude
28                    issues
29              issue_labels
30         issue_message_ref
31              issue_events
32           issue_assignees
33            

In [4]:
desc_table_query = f"""
SELECT
   COLUMN_NAME
FROM
   information_schema.COLUMNS
WHERE
   TABLE_NAME = 'repo_groups';
    """
table_desc = pd.read_sql_query(desc_table_query, con=engine)
print(table_desc)


             column_name
0          repo_group_id
1                rg_name
2         rg_description
3             rg_website
4             rg_recache
5       rg_last_modified
6                rg_type
7            tool_source
8           tool_version
9            data_source
10  data_collection_date


In [2]:
get_id_query = f"""
SELECT
   rg_name, repo_group_id
FROM
   repo_groups;
    """
get_id_desc = pd.read_sql_query(get_id_query, con=engine)
print(get_id_desc)

                      rg_name  repo_group_id
0          Pivotal-DataFabric          25162
1                 SteeltoeOSS          25161
2               spring-guides          25160
3                greenplum-db          25159
4                       GemXD          25158
5             Spring-projects          25157
6                   spring-io          25156
7                    rabbitmq          25155
8                 pivotal-gss          25176
9           pivotal-education          25175
10            pivotalsoftware          25174
11                 Pivotal-sg          25173
12                    pivotal          25172
13            pivotalservices          25171
14  Pivotal-Field-Engineering          25170
15                      Rails              1
16  platform-acceleration-lab          25169
17            cf-platform-eng          25168
18           pivotal-cloudops          25167
19               pcfdev-forks          25166
20             pivotaltracker          25165
21        